# การสร้างด้วยโมเดล Mistral

## บทนำ

บทเรียนนี้จะครอบคลุม:
- การสำรวจโมเดล Mistral แบบต่างๆ
- ทำความเข้าใจกรณีการใช้งานและสถานการณ์ที่เหมาะสมของแต่ละโมเดล
- ตัวอย่างโค้ดที่แสดงจุดเด่นเฉพาะของแต่ละโมเดล


## โมเดล Mistral

ในบทเรียนนี้ เราจะมาดูโมเดล Mistral ทั้ง 3 แบบ ได้แก่ **Mistral Large**, **Mistral Small** และ **Mistral Nemo**

โมเดลเหล่านี้สามารถใช้งานได้ฟรีบนตลาดโมเดลของ Github โค้ดในสมุดบันทึกนี้จะใช้โมเดลเหล่านี้ในการรันโค้ด คุณสามารถดูรายละเอียดเพิ่มเติมเกี่ยวกับการใช้ Github Models เพื่อ [สร้างต้นแบบด้วยโมเดล AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)


## Mistral Large 2 (2407)
Mistral Large 2 เป็นโมเดลเรือธงรุ่นล่าสุดจาก Mistral ที่ออกแบบมาสำหรับการใช้งานในองค์กร

โมเดลนี้เป็นการอัปเกรดจาก Mistral Large รุ่นแรก โดยมีจุดเด่นดังนี้
-  ขยายขนาด Context Window ให้ใหญ่ขึ้น - 128k เทียบกับ 32k
-  ประสิทธิภาพที่ดีกว่าในงานคณิตศาสตร์และการเขียนโค้ด - ความแม่นยำเฉลี่ย 76.9% เทียบกับ 60.4%
-  รองรับหลายภาษาได้ดีขึ้น - ภาษาที่รองรับ ได้แก่ อังกฤษ, ฝรั่งเศส, เยอรมัน, สเปน, อิตาลี, โปรตุเกส, ดัตช์, รัสเซีย, จีน, ญี่ปุ่น, เกาหลี, อาหรับ และฮินดี

ด้วยคุณสมบัติเหล่านี้ Mistral Large จึงโดดเด่นในด้าน
- *Retrieval Augmented Generation (RAG)* - ด้วย Context Window ที่ใหญ่ขึ้น
- *Function Calling* - โมเดลนี้รองรับการเรียกใช้ฟังก์ชันโดยตรง สามารถเชื่อมต่อกับเครื่องมือหรือ API ภายนอกได้ การเรียกใช้ฟังก์ชันสามารถทำได้ทั้งแบบขนานหรือแบบเรียงลำดับทีละขั้นตอน
- *Code Generation* - โมเดลนี้ทำได้ดีเยี่ยมในการสร้างโค้ดภาษา Python, Java, TypeScript และ C++


ในตัวอย่างนี้ เราใช้ Mistral Large 2 เพื่อรันรูปแบบ RAG กับเอกสารข้อความ คำถามถูกเขียนเป็นภาษาเกาหลีและถามเกี่ยวกับกิจกรรมของผู้เขียนก่อนเข้าเรียนมหาวิทยาลัย

เราใช้ Cohere Embeddings Model เพื่อสร้าง embedding ของเอกสารข้อความรวมถึงคำถามด้วย สำหรับตัวอย่างนี้ เราใช้แพ็กเกจ faiss ของ Python เป็น vector store

prompt ที่ส่งไปยังโมเดล Mistral จะรวมทั้งคำถามและส่วนของข้อความที่ค้นพบว่าคล้ายกับคำถาม โมเดลจะให้คำตอบในรูปแบบภาษาธรรมชาติ


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small เป็นอีกหนึ่งโมเดลในตระกูล Mistral ที่อยู่ในกลุ่ม premier/enterprise ตามชื่อของมัน โมเดลนี้เป็น Small Language Model (SLM) ข้อดีของการใช้ Mistral Small ได้แก่:
- ประหยัดค่าใช้จ่ายเมื่อเทียบกับ Mistral LLMs เช่น Mistral Large และ NeMo - ราคาถูกลงถึง 80%
- หน่วงเวลาต่ำ - ตอบสนองได้รวดเร็วกว่า LLMs ของ Mistral
- ยืดหยุ่น - สามารถนำไปใช้งานในสภาพแวดล้อมที่หลากหลาย โดยมีข้อจำกัดด้านทรัพยากรน้อยกว่า

Mistral Small เหมาะสำหรับ:
- งานที่เกี่ยวกับข้อความ เช่น สรุปเนื้อหา วิเคราะห์ความรู้สึก และแปลภาษา
- แอปพลิเคชันที่มีการร้องขอบ่อย ๆ เพราะคุ้มค่าเรื่องต้นทุน
- งานโค้ดที่ต้องการความรวดเร็ว เช่น ตรวจสอบโค้ดและแนะนำโค้ด


## เปรียบเทียบ Mistral Small และ Mistral Large

เพื่อแสดงความแตกต่างของเวลาแฝงระหว่าง Mistral Small และ Large ให้รันเซลล์ด้านล่างนี้

คุณจะเห็นความแตกต่างของเวลาตอบสนองประมาณ 3-5 วินาที และสังเกตความยาวของคำตอบรวมถึงสไตล์ที่แตกต่างกันเมื่อใช้พรอมต์เดียวกัน


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

เมื่อเทียบกับโมเดลอีกสองตัวที่พูดถึงในบทเรียนนี้ Mistral NeMo เป็นโมเดลเดียวที่ใช้ฟรีและมีไลเซนส์แบบ Apache2

โมเดลนี้ถือเป็นการอัปเกรดจาก LLM แบบโอเพ่นซอร์สรุ่นก่อนของ Mistral ที่ชื่อว่า Mistral 7B

คุณสมบัติอื่น ๆ ของโมเดล NeMo ได้แก่

- *การแปลงโทเคนที่มีประสิทธิภาพมากขึ้น:* โมเดลนี้ใช้ตัวแปลงโทเคน Tekken แทนที่จะใช้ tiktoken ที่นิยมมากกว่า ซึ่งช่วยให้รองรับภาษาและโค้ดได้หลากหลายและมีประสิทธิภาพมากขึ้น

- *การปรับแต่งเพิ่มเติม (Finetuning):* โมเดลพื้นฐานสามารถนำไปปรับแต่งเพิ่มเติมได้ ทำให้มีความยืดหยุ่นมากขึ้นสำหรับกรณีการใช้งานที่ต้องการการปรับแต่ง

- *การเรียกใช้ฟังก์ชันโดยตรง (Native Function Calling)* - เช่นเดียวกับ Mistral Large โมเดลนี้ได้รับการฝึกฝนให้รองรับการเรียกใช้ฟังก์ชันโดยตรง ซึ่งถือเป็นหนึ่งในโมเดลโอเพ่นซอร์สรุ่นแรก ๆ ที่มีความสามารถนี้


## Mistral NeMo

เมื่อเทียบกับโมเดลอีกสองตัวที่พูดถึงในบทเรียนนี้ Mistral NeMo เป็นโมเดลเดียวที่ใช้ฟรีและมีไลเซนส์แบบ Apache2

โมเดลนี้ถือเป็นการอัปเกรดจาก LLM แบบโอเพ่นซอร์สรุ่นก่อนของ Mistral ที่ชื่อว่า Mistral 7B

คุณสมบัติอื่น ๆ ของโมเดล NeMo ได้แก่

- *การแปลงโทเคนที่มีประสิทธิภาพมากขึ้น:* โมเดลนี้ใช้ตัวแปลงโทเคน Tekken แทนที่จะใช้ tiktoken ที่นิยมมากกว่า ซึ่งช่วยให้รองรับภาษาและโค้ดได้หลากหลายและมีประสิทธิภาพมากขึ้น

- *การปรับแต่งเพิ่มเติม (Finetuning):* โมเดลพื้นฐานสามารถนำไปปรับแต่งเพิ่มเติมได้ ทำให้มีความยืดหยุ่นมากขึ้นสำหรับกรณีการใช้งานที่ต้องการการปรับแต่ง

- *การเรียกใช้ฟังก์ชันโดยตรง (Native Function Calling)* - เช่นเดียวกับ Mistral Large โมเดลนี้ถูกฝึกมาให้รองรับการเรียกใช้ฟังก์ชันโดยตรง ซึ่งถือเป็นหนึ่งในโมเดลโอเพ่นซอร์สรุ่นแรก ๆ ที่มีความสามารถนี้


### การเปรียบเทียบ Tokenizer

ในตัวอย่างนี้ เราจะมาดูกันว่า Mistral NeMo จัดการกับการแยกคำ (tokenization) อย่างไรเมื่อเทียบกับ Mistral Large

ทั้งสองตัวอย่างใช้ prompt เดียวกัน แต่คุณจะเห็นว่า NeMo ส่งคืนจำนวนโทเคนน้อยกว่าเมื่อเทียบกับ Mistral Large


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## การเรียนรู้ไม่ได้หยุดแค่นี้ เดินหน้าต่อไป

หลังจากจบบทเรียนนี้แล้ว ลองเข้าไปดู [ชุดการเรียนรู้ Generative AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ของเรา เพื่อพัฒนาความรู้ด้าน Generative AI ให้มากขึ้น!



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
